In [1]:
import torch
import numpy as np
import pandas as pd
import random as rd
import csv

In [2]:
# default stuff
dtype = torch.double
device = torch.device("cpu")

# architecture definition
N_in, N_hid, N_out = 2, 10, 4
batch_size = 200
learning_rate = 1e-6

#weight matrix
w1 = torch.randn(N_in, N_hid, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(N_hid, N_out, device=device, dtype=dtype, requires_grad=True)

In [3]:
# data manipulation
# ho cambiato il file così ora sono 0 e 1 i colori
data = pd.read_csv('/home/burrito/Documents/python/pytorch/green_red/data_train_4.csv')
test = pd.read_csv('/home/burrito/Documents/python/pytorch/green_red/data_test_4.csv')
data_np_x = data[['x', 'y']].to_numpy()
data_np_y = data[['color']].to_numpy()
data_test = test.to_numpy()

In [4]:
batchesX = np.asarray(np.array_split(data_np_x, 100))
batchesY = np.asarray(np.array_split(data_np_y, 100))

In [5]:
Xtr = torch.from_numpy(batchesX[2])
Ytr = torch.from_numpy(batchesY[2])
Xtest = torch.from_numpy(data_test)

In [6]:
for epoch in range(100000):
    b = rd.randint(0, 99)
    Xtr = torch.from_numpy(batchesX[b])
    Ytr = torch.from_numpy(batchesY[b])
    y = Xtr.mm(w1).clamp(min=0).mm(w2)
    # ho provato a fare la cross-entropy ma non va, la 
    # la loss viene sempre uguale
    s = torch.gather(y, 1, Ytr) # gather serve a indicizzare y in base a Ytr
    loss = ((-((s.exp())/(y.exp().sum())).log())).sum()
    # questo valore qui sotto è a caso ma dovrebbe essere circa la lunghezza
    # di una batch
    loss = loss/100
    if epoch%10000 == 0:
        print(epoch, loss.item())
    loss.backward()
    with torch.no_grad():
        w1 -= learning_rate*w1.grad
        w2 -= learning_rate*w2.grad
        w1.grad.zero_()
        w2.grad.zero_()

0 8.815787759193213
10000 8.689655124594147
20000 9.076441298993608
30000 8.595716133678977
40000 8.946040627723717
50000 8.822356147872082
60000 8.63889009260031
70000 8.18081907854684
80000 8.596850367193728
90000 8.32880976093098


In [7]:
prediction = Xtest.mm(w1).clamp(min=0).mm(w2)

tensor([0.9929, 0.1523], dtype=torch.float64)


In [10]:
pred = open("/home/burrito/Documents/python/pytorch/green_red/pred_4.csv", 'w+', newline='')
writer = csv.writer(pred)
writer.writerow(["color"])
for i in prediction:
    value, index = torch.max(i, 0)
    writer.writerow([index])